## **Конспект**

**Как происходит моделирование:**

*   **Подготовка данных:** import — загрузка данных, print — вывод данных, x.isnull().sum — проверка наличия пропусков, разбивка выборки на тестовую и обучающую.
*   **Обучение модели:** метод LinearRegression — линейная регрессия, GradientBoostingRegressor — градиентный бустинг.
*   **Валидация модели:** чем меньше результат, тем лучше.
*   **Внедрение:** модель оборачивается в сервис или контейнер.

# Как происходит моделирование

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Определение бизнес-требований

На этом этапе мы определяем, что именно нужно сделать и зачем это нужно сделать.  
Такая постановка вопроса помогает понять, нужно ли вообще применять методы машинного обучения в данном конкретном случае.  
После этого мы запрашиваем необходимые данные и начинаем работу. При необходимости мы сможем вернуться на этот шаг.

## Подготовка данных

Загрузим данные

In [5]:
#%pip install scikit-learn==1.0
#setuptools



In [2]:
from sklearn.datasets import load_boston
dataset = load_boston()

C:\PythonProject\skillfactory мат анализ\.venv1\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this case special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the 

In [3]:
dataset.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [4]:
X = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
y = pd.Series(dataset['target'])

In [5]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [6]:
print(dataset['DESCR'][:1220])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Проверим наличие пропусков

In [7]:
X.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

Посмотрим на корреляцию признаков с целевой переменной:

In [7]:
trimmed_feature_names = []
for i in range(X.shape[1]):
    correlation = np.corrcoef(X.iloc[:, i], y)[0, 1]
    if abs(correlation) > 0.5:
        feature_name = dataset['feature_names'][i]
        print(feature_name, correlation)
        trimmed_feature_names.append(feature_name)

RM 0.6953599470715393
PTRATIO -0.5077866855375616
LSTAT -0.7376627261740151


In [8]:
X_trimmed = X[trimmed_feature_names]
# X = X_trimmed

### Разбиение выборки

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (404, 13)
X_test: (102, 13)
y_train: (404,)
y_test: (102,)


## Обучение модели

 ### Линейная регрессия

In [14]:
from sklearn.linear_model import LinearRegression
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

### Градиентный бустинг

In [17]:
from sklearn.ensemble import GradientBoostingRegressor
predictor = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01)
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

## Валидация модели

In [18]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

1.982145634759136

## Внедрение

В данном курсе мы почти не касаемся внедрения, но нужно помнить, что только после выполнения этого шага модель сможет приносить пользу.  
На этом шаге обычно мы оборачиваем модель в сервис или в контейнер.